In [3]:
# coding:utf-8
"""
本篇为tag_age_from_song()的调试笔记，后期将加入user_all()
"""
import re
import pandas as pd
import numpy as np
from impala.dbapi import connect
from impala.util import as_pandas
import pymysql.cursors
from datetime import datetime

# Global variables
mysql_host = '172.21.1.151'
mysql_port = 3306
mysql_user = 'xli'
mysql_password = '123456'
mysql_db = 'kdwtemp2'
mysql_charset = 'utf8mb4'

def easy_print(tname, records=10):
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()     
    
    sql = 'SELECT * FROM ' + tname + ' LIMIT 10' 
    cur.execute(sql)
    read_in = as_pandas(cur)    
    cur.close()
    conn.close()
    
    #print in a format
    columns = read_in.columns.values
    values = read_in.values
    records = []
    for v in values:
        records.append(zip(columns, v))
    
    for ix,r in enumerate(records):
        print('----------第%s条记录--------' % (ix+1))
        for k, v in r:
            print('%s: %s' % (k, v))

def connect_Mysql(sql_columns_select,tname): #连接Mysql数据库
    mysqlConnection = pymysql.connect(host=mysql_host,
                                      port=mysql_port,
                                      user=mysql_user,
                                      password=mysql_password,
                                      db=mysql_db,
                                      charset=mysql_charset)
                                      #cursorclass=pymysql.cursors.DictCursor)
    try:
        sql = 'SELECT '+ sql_columns_select + ' FROM '+ tname #+ ' LIMIT 100'
        df = pd.read_sql(sql,mysqlConnection)
        return df
    except:
        mysqlConnection.close()
        
def year_stage(data): #给年份分段
    if data >= 2000:
        return 5
    elif data >= 1990 and data < 2000:
        return 4
    elif data >= 1980 and data < 1990:
        return 3
    elif data >= 1970 and data < 1980:
        return 2
    else:
        return 1 
    
def song_release_time(): #抽取出songid对应的歌曲发行时间
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()
    sql = """
    SELECT songid,releasetime FROM kdw_tbl_song_info"""
    cur.execute(sql)
    df = as_pandas(cur)
    # 清洗掉不符合时间格式的数据行
    pattern = re.compile(r"^\d{4}-\d{2}-\d{2}$")
    df = df[map(lambda x: re.match(pattern,x) is not None,df['releasetime'])]
    #提取出年份
    df['releasetime']=pd.to_datetime(df['releasetime'], format='%Y-%m-%d')
    df['year'] = [i.year for i in df['releasetime']]  
    #通过歌曲release time判断发行时间年龄段
    df['age_year'] = [year_stage(i) for i in df['year']]
    df.drop('releasetime',axis=1,inplace=True)
    cur.close()
    conn.close()  
    return df

def read_play_cnt(): #抽取出用户点歌记录,并对点歌次数进行统计
    conn = connect(host='172.21.1.120', port=21050, user='udms', database='kmi_month') 
    cur = conn.cursor()
#     sql = """
#     SELECT customerid,songid,playcount as song_play_cnt FROM kdw_tbl_song_menu_event_statics """
    sql = """
    SELECT customerid,songid,sum(playcount) as song_play_cnt FROM kdw_tbl_song_menu_event_statics
    GROUP BY customerid,songid
    ORDER BY customerid,sum(playcount) DESC"""
    cur.execute(sql)
    df = as_pandas(cur)
    # 只保留每个用户点歌次数最多的歌
    df.drop_duplicates('customerid',inplace=True)    
    #后续需要与用户表做表连
    cur.close()
    conn.close()    
    return df    

def language_stage(data):
    if data == "粤语".decode("utf-8"):
        return 1
    elif data == "国语".decode("utf-8"):
        return 2
    else:
        return 3
    
def age_song_language():
    df = connect_Mysql('SongID as songid,LanguageType','kdw_tbl_song')
    df['age_lan'] = [language_stage(i) for i in df['LanguageType']]
    df.drop('LanguageType',axis=1,inplace=True)
    return df

def tag_age_from_song():
    # 1. 抽取出songid对应的歌曲发行时间并打上时间段标签
    df_year = song_release_time()
    # 2. 对用户点歌记录统计操作
    df_play_cnt = read_play_cnt()
    tmp_df = pd.merge(df_play_cnt,df_year,how='left',on='songid')
    # 3. 根据歌曲语言进行语言标签操作
    df_lan = age_song_language()
    # 4. 合并所有表
    total_df = pd.merge(tmp_df,df_lan,how='left',on='songid')
    
    # 将中间结果暂存于Mysqldb中
    conn = pymysql.connect(host=mysql_host,
                                      port=mysql_port,
                                      user=mysql_user,
                                      password=mysql_password,
                                      db=mysql_db,
                                      charset=mysql_charset)
    
    total_df.to_sql(name="ljl_tag_age_from_song", con=conn, flavor='mysql', if_exists='replace', index=False)
    conn.close()

In [4]:
tag_age_from_song()

D:\Users\Lenovo\Anaconda2\lib\site-packages\pandas\core\generic.py:1003: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  dtype=dtype)


# 1. 抽取出songid对应的歌曲发行时间并打上时间段标签

In [11]:
df = song_release_time()

In [8]:
df.head()

,songid,releasetime,year,age_year
1,00000015,1999-10-01,1999,4
2,00000021,2004-11-12,2004,5
3,00000029,2014-06-03,2014,5
4,00000030,2013-03-20,2013,5
5,00000037,2013-12-10,2013,5


In [9]:
df.shape

(27938, 4)

### drop掉原始发行时间

In [12]:
df.head()

,songid,year,age_year
1,00000015,1999,4
2,00000021,2004,5
3,00000029,2014,5
4,00000030,2013,5
5,00000037,2013,5


# 2. 对用户点歌记录统计操作

In [13]:
df = read_play_cnt()

### 读出数据-这部分数据量比较大

In [14]:
df.head()

,customerid,songid,song_play_cnt
0,8673,72903095,1
1,3835,00002612,1
2,233,00006488,2
3,9019,20000921,1
4,6,10621,1


In [15]:
df.shape

(10532486, 3)

In [16]:
pd.value_counts_counts_counts(df['customerid'])

6        308503
-1        48694
5         44186
4         43842
8050      31217
5053      28676
1872      24411
2871      17817
2874      16482
5677      15146
5862      14967
7996      14384
1004      14219
9258      14081
9450      13802
5742      13753
7460      13351
1848      13188
4050      13128
5555      13070
7586      12692
1073      12486
5887      12452
5586      12335
2634      12189
2054      12039
5035      11639
1836      11587
4242      11562
390       11542
          ...  
8544          2
7297          2
10650         2
4659          2
5782          1
8554          1
44            1
3429          1
1117          1
10619         1
2917          1
27            1
10244         1
403           1
10350         1
4429          1
8980          1
8846          1
0             1
10327         1
9691          1
2894          1
5625          1
9150          1
2308          1
9406          1
4238          1
860           1
3935          1
8747          1
Name: customerid, dtype:

## 数据量太大了，使用dataframe进行操作太慢，换用sql操作

In [36]:
df = read_play_cnt()

In [37]:
df.head()

,customerid,songid,song_play_cnt
0,-1,00006488,389
1,-1,00041123,264
2,-1,00024830,247
3,-1,00022830,197
4,-1,13015848,171


In [38]:
df.shape

(5832828, 3)

In [41]:
pd.value_counts(df['song_play_cnt'])

1       3703922
2       1015838
3        434950
4        221830
5        129075
6         83040
7         55774
8         39663
9         28778
10        21452
11        16472
12        13020
13        10250
14         8247
15         6730
16         5714
17         4570
18         3867
19         3286
20         2752
21         2478
22         1996
23         1731
24         1525
25         1433
26         1205
27         1053
28          960
29          796
30          738
         ...   
398           1
394           1
392           1
386           1
380           1
379           1
374           1
370           1
358           1
355           1
349           1
343           1
340           1
338           1
337           1
336           1
335           1
334           1
331           1
327           1
325           1
318           1
317           1
312           1
298           1
296           1
293           1
292           1
291           1
2310          1
Name: song_play_cnt, dty

## 保留每个用户点歌次数最多的歌

In [43]:
df.drop_duplicates('customerid',inplace=True)

In [44]:
df.shape

(6734, 3)

In [45]:
df.head()

,customerid,songid,song_play_cnt
0,-1,00006488,389
16720,0,00022539,1
16721,10002,00090974,11
16822,10004,00022830,1
16837,10006,14001176,4


# 3. 根据歌曲语言进行语言标签操作

In [47]:
df = age_song_language()

In [48]:
df.head()

,songid,age_lan
0,00000002,3
1,00000003,3
2,00000004,3
3,00000005,3
4,00000006,3


In [49]:
df.shape

(157091, 2)

# 4. 合并所有表

In [50]:
df = tag_age_from_song()

In [51]:
df.head()

,customerid,songid,song_play_cnt,year,age_year,age_lan
0,-1,00006488,389,NaN,NaN,2
1,0,00022539,1,NaN,NaN,1
2,10002,00090974,11,2011,5,2
3,10004,00022830,1,2004,5,2
4,10006,14001176,4,NaN,NaN,2


In [52]:
df.shape

(6734, 6)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6734 entries, 0 to 6733
Data columns (total 6 columns):
customerid       6734 non-null object
songid           6734 non-null object
song_play_cnt    6734 non-null int64
year             2305 non-null float64
age_year         2305 non-null float64
age_lan          6618 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 368.3+ KB


In [54]:
df.columns

Index([u'customerid', u'songid', u'song_play_cnt', u'year', u'age_year',
       u'age_lan'],
      dtype='object')